In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('/home/prashant/incedo/test/Dataset2147b1d/train_file.csv')
df_train['source_'] = 1
df_test = pd.read_csv('/home/prashant/incedo/test/Dataset2147b1d/test_file.csv')
df_test['source_'] =0

In [ ]:
df = pd.concat([df_train,df_test])
df.reset_index(inplace=True)

In [ ]:
df.head(3)
df['GeoLocation'].fillna(df['GeoLocation'].mode()[0], inplace=True)
df['GeoLocation'].value_counts()

In [ ]:
#useless columns [patient_id,year,locDesc,Geoloc,Greater_Risk_Question,Description]
geo_data = list(set(list(dict(df['GeoLocation'].value_counts()).keys())))

In [ ]:
vec = []
for i in geo_data:
    i  = eval(i)
    x = i[0]
    y = i[1]
    vec.append((x,y))

In [ ]:
from pprint import pprint
X = np.array(vec)
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=15, random_state=0).fit(X)
labels = (kmeans.labels_)
geo_clus_dict = {}
for cnt,i in enumerate(geo_data):
    geo_clus_dict[i] = labels[cnt]

In [ ]:
#geo_clus_dict['GeneratorExit']
df['GeoLocation'][1]

In [ ]:
new_col = []
for i in range(df.shape[0]):
    clus_no = geo_clus_dict[df['GeoLocation'][i]]
    new_col.append(clus_no)

In [ ]:
df['geo_loc_clu_no'] = new_col

In [ ]:
new_df = df.drop(['Patient_ID','YEAR','LocationDesc','Greater_Risk_Question', 'Description','GeoLocation'],axis=1)
len(set(df['QuestionCode']))


In [ ]:
print(new_df.isnull().sum())
new_df.head()

In [ ]:
new_df.dtypes

In [ ]:
col_to_be_converted = ['Subtopic','Sex','Race','Grade','QuestionCode',\
                       'StratID1','StratID2','StratID3','StratificationType']
one_hot_vector_df = pd.get_dummies(new_df,columns=col_to_be_converted)

In [ ]:
new_df.head()

In [ ]:
new_df['Sex'].value_counts()

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
new_col = np.array(new_df['Sample_Size']).reshape(-1,1)
scaled_df = list(map(lambda x : x[0] ,scaler.fit_transform(new_col)))


In [ ]:
one_hot_vector_df['Sample_Size'] = scaled_df 

In [ ]:
one_hot_vector_df.head()

In [ ]:
preprocessed_main_test = one_hot_vector_df.loc[one_hot_vector_df['source_'] == 0]
preprocessed_main_train = one_hot_vector_df.loc[one_hot_vector_df['source_'] == 1]

In [ ]:
preprocessed_main_test.drop(['source_','Greater_Risk_Probability'],inplace=True,axis =1)
preprocessed_main_train.drop(['source_'],inplace=True,axis =1)

In [ ]:
print(preprocessed_main_test.shape)
preprocessed_main_train.shape
preprocessed_main_test

In [ ]:
y_train = (preprocessed_main_train['Greater_Risk_Probability'])
x_train = preprocessed_main_train.drop(['Greater_Risk_Probability'],axis =1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,test_size =.1)

In [ ]:
# # Import the model we are using
# from sklearn.ensemble import RandomForestRegressor

# # Instantiate model with 1000 decision trees
# # rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

# # # Train the model on training data
# # rf.fit(x_train, y_train)

# params = {
# 'num_leaves': 15,
# 'max_bin': 100,
# 'min_data_in_leaf': 45,
# 'learning_rate': 0.01,
# 'min_sum_hessian_in_leaf': 0.000446,
# 'bagging_fraction': 0.55,
# 'bagging_freq': 5,
# 'max_depth': 14,
# 'save_binary': True,
# 'seed': 31452,
# 'feature_fraction_seed': 31415,
# 'feature_fraction': 0.51,
# 'bagging_seed': 31415,
# 'drop_seed': 31415,
# 'data_random_seed': 31415,
# #'objective': 'multiclass',
# 'boosting_type': 'gbdt',
# 'verbose': 1,
# 'metric': 'rmse',
# #'is_unbalance': False,
# #     'categorical_feature': "name:branch_id"
# # 'scale_pos_weight': len(train_df["target"] == 0) / sum(train_df["target"])
# }

# # Import the model we are using
# #from sklearn.ensemble import RandomForestClassifier
# # from tpot import TPOTClassifier
# import lightgbm as lgb

# d_train = lgb.Dataset(X_train, label=y_train_internal)
# # Instantiate model with 1000 decision trees
# #rf = RandomForestClassifier()

# # Train the model on training data
# #rm = rf.fit(X_train, y_train_internal)
# rf = lgb.train(params, d_train,20000)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

from subprocess import check_output
from datetime import time
import numpy as np
# Establish model
model = RandomForestRegressor(n_jobs=-1)

estimators = np.arange(10, 200, 10)
scores = []
for n in estimators:
    model.set_params(n_estimators=n)
    model.fit(X_train, y_train_internal)
    scores.append(model.score(X_test, y_test_internal))

model.set_params(n_estimators=170)
model.fit(X_train, y_train_internal)

In [ ]:
pred_internal = model.predict(preprocessed_main_test)

In [ ]:
id_ = list(df_test['Patient_ID'])
len(id_)

In [ ]:
final_res = pd.DataFrame()
final_res['Patient_ID'] = id_
final_res['Greater_Risk_Probability'] = pred_internal

In [ ]:
final_res.to_csv('lgb_1_cluster_app_clus_15_rf_regressor.csv',index=False)